<a href="https://colab.research.google.com/github/Anas321/ImageCaptioning/blob/master/Image_captioning_InceptV3_Flickr_data_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
pwd

'/content'

In [ ]:
cd /content/drive/My Drive/Colab Notebooks/ImageCaptioning/InceptV3/

/content/drive/My Drive/Colab Notebooks/ImageCaptioning/InceptV3


In [ ]:
from google.colab import files

In [ ]:
# uploaded = files.upload()

In [ ]:
ls

annotations/
cleaned_descriptions.json
features_for_all_images_Flickr_backup.pkl
features_for_all_images_Flickr_NEW.pkl
features_for_all_images_Flickr.pkl
features_for_all_images_Inception_MSCOCO.pkl
Flicker8k_Dataset/
Flickr8k_text/
Image_captioning_inceptV3_Flickr_main.ipynb
Image_captioning_inceptV3_MSCOCO.ipynb
__MACOSX/
model.png
MODELS/
tokenizer.pkl
train2014/


In [ ]:
EXTRACT_IMAGES_FEATURES = 0

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
# import collections
import random
import re
import numpy as np
import os
import time
import json
from glob import glob
from PIL import Image
import pickle
from tqdm import tqdm

from os import listdir
from pickle import dump
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import Model



**Extract Images Features**

In [ ]:
# Extract images features
images_folder_train = 'Flicker8k_Dataset'
all_images_features_file_name = 'features_for_all_images_Flickr_NEW.pkl'

# extract features from each image
def extract_images_features(directory):
    # load the VGG16 model
    model = InceptionV3()
    # re-structure the model
    model = Model(inputs=model.inputs, outputs=model.layers[-2].output)
    # print model summary
    print(model.summary())
    # define all_features dictionery to store 
    # features for all images
    all_features = dict()
    #counter = 1
    for name in tqdm(listdir(directory)):
        # get an image from a file
        filename = directory + '/' + name
        #print(filename)
        image = load_img(filename, target_size=(299,299))
        # convert the image pixles to a numpy array
        image = img_to_array(image)
        #print(image.shape)
        # reshape the created numpy array for the model
        image = image.reshape(1, image.shape[0], image.shape[1], image.shape[2])
        #print(image.shape)
        # prepare the image for the VGG16 model
        image = preprocess_input(image)
        # get features
        feature = model.predict(image, verbose=0)
        # get image id
        image_name = name.split('.')[0]
        #print(image_name)
        # store feature in the features dictionery
        all_features[image_name] = feature
        #print(counter,'>', name)
        #counter += 1
    return all_features

if EXTRACT_IMAGES_FEATURES == 1:
    # extract features from all images
    directory = images_folder_train
    features = extract_images_features(directory)
    print('Size of extracted features:', len(features))
    # save to file
    with open(all_images_features_file_name, 'wb') as f:
        pickle.dump(features, f)
#     dump(features, open('features_for_all_images_Inception_MSCOCO.pkl', 'wb'))

**Parse Text Data**

In [ ]:
ls

annotations/
cleaned_descriptions.json
features_for_all_images_Flickr_backup.pkl
features_for_all_images_Flickr_NEW.pkl
features_for_all_images_Flickr.pkl
features_for_all_images_Inception_MSCOCO.pkl
Flicker8k_Dataset/
Flickr8k_text/
Image_captioning_inceptV3_Flickr_main.ipynb
Image_captioning_inceptV3_MSCOCO.ipynb
__MACOSX/
model.png
MODELS/
tokenizer.pkl
train2014/


In [ ]:
# def parse_text_file(filename):

percentage_of_selected_images = 1.0
filename = 'Flickr8k_text/Flickr8k.token.txt'
with open(filename, 'r') as f:
  file_content = f.read()
# Create a dictionary where keys are the images ids and values are the captions
desc_dict = dict()
for line in file_content.split('\n'):
  segments = line.split()
  if len(segments) < 2:
    continue
  image_id = segments[0]
  image_desc = segments[1:]
  image_desc = ' '.join(image_desc)
  if image_id not in desc_dict.keys():
    desc_dict[image_id] = list()
  desc_dict[image_id].append(image_desc)
# select only percentage of the available images (for memory restrictions)
if percentage_of_selected_images < 1.0:
  num_of_selected_images = round(len(desc_dict.keys())*percentage_of_selected_images)
  desc_dict = list(desc_dict.items())[0:num_of_selected_images]
  desc_dict = dict(desc_dict)
# Clean desc_dict
for key, desc_list in desc_dict.items():
  # Loop through the list for each key
  for i in range(len(desc_list)):
    current_desc = desc_list[i]
    # Tokenize; i.e., split
    current_desc = current_desc.split()
    # Lower case all words
    current_desc = [word.lower() for word in current_desc]
    # Remove any words with length <= 1
    current_desc = [word for word in current_desc if len(word)>1]
    # Remove any puctuations
    for j in range(len(current_desc)):
      current_word = current_desc[j]
      if not current_word.isalnum():
        letters = list(current_word)
        letters = [letter for letter in letters if letter not in string.punctuation]
        current_desc[j] = ''.join(letters)
    # Remove any words happen to contain numbers in them
    current_desc = [word for word in current_desc if word.isalpha()]
    # Join the splitted words together again
    current_desc = ' '.join(current_desc)
    # Add startseq and endseq
    current_desc = 'startseq ' + current_desc + ' endseq'
    # Assain it back to the dictionery for its corresponding key
    desc_list[i] = current_desc
    # Print length of desc_dict
desc_dict.pop('2258277193_586949ec62')
print('There are {:d} images.'.format(len(desc_dict)))
# Save the cleaned descriptions as a json file
with open('cleaned_descriptions.json', 'w') as f:
  json.dump(desc_dict, f)
print('\nCleaned descriptions are saved in \'cleaned_descriptions.json\' file.')

There are 8091 images.

Cleaned descriptions are saved in 'cleaned_descriptions.json' file.


In [ ]:
# desc_dict.pop('2258277193_586949ec62')

['startseq people waiting for the subway endseq',
 'startseq some people looking out windows in large building endseq',
 'startseq three people are waiting on train platform endseq',
 'startseq three people standing at station endseq',
 'startseq two woman and one man standing near train tracks endseq']

In [ ]:
desc_dict['2258277193_586949ec62']

KeyError: ignored

In [ ]:
ll

total 999106
drwx------ 2 root      4096 Nov  4 22:05 annotations/
-rw------- 1 root   3047050 Nov 23 02:00 cleaned_descriptions.json
-rw------- 1 root 133296868 Nov 18 18:35 features_for_all_images_Flickr_backup.pkl
-rw------- 1 root  67015398 Nov 18 20:02 features_for_all_images_Flickr_NEW.pkl
-rw------- 1 root 133296868 Nov 16 17:57 features_for_all_images_Flickr.pkl
-rw------- 1 root 686187837 Nov  3 17:40 features_for_all_images_Inception_MSCOCO.pkl
drwx------ 2 root      4096 Nov 17 01:49 Flicker8k_Dataset/
drwx------ 2 root      4096 Nov 17 01:51 Flickr8k_text/
-rw------- 1 root     54178 Nov 23 01:55 Image_captioning_inceptV3_Flickr_main.ipynb
-rw------- 1 root     67040 Nov 15 19:57 Image_captioning_inceptV3_MSCOCO.ipynb
drwx------ 2 root      4096 Nov 17 01:49 __MACOSX/
-rw------- 1 root     66172 Nov 23 01:10 model.png
drwx------ 2 root      4096 Nov  5 20:22 MODELS/
-rw------- 1 root     25893 Nov 23 01:06 tokenizer.pkl
drwx------ 2 root      4096 Nov  5 17:40 train2014/
